In [86]:
import sys
sys.path.insert(0, '..')
print(sys.path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
print(sys.path)
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import json
from autofaiss import build_index


['..', '..', '..', '..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.ipython', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/tmp/tmp9pbbnyjp', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model']
['..', '..', '..', '..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/l

In [69]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv")
clean_df.head()

,caption,product,label,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num,step_5_no_dupe,step_6_no_len_1_token,caption_crop_first_70%,caption_crop_last_70%,caption_crop_middle_70%,captions_cn
0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg,32820,tanabata valentine's day shenzhen bao'an m nan...,tanabata valentines day shenzhen baoan m nansh...,tanabata valentines day shenzhen baoan nanshan...,tanabata valentines day shenzhen baoan nanshan...,luohu arrangement valentines decorative chest ...,arrangement chest valentines decorative nansha...,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,M Nanshan Futian Luohu Car Decorative Flowers ...,Shenzhen Bao'an M Nanshan Futian Luohu Car Dec...,塔那巴塔情人节 清心宝安 南山福田罗虎 汽车装饰花花花花花花 胸衣布置安排
1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg,8326,children's toys little girl over 6 years old g...,childrens toys little girl over 6 years old gi...,childrens toys little girl 6 years old girls 3...,childrens toys little girl years old girls boy...,years childrens little old boys baby gift chil...,years childrens little old boys baby gift chil...,Children's Toys Little Girl over 6 Years Old G...,Old Girls 3-9 Boys 7 Students I Day Gift 10 Ch...,over 6 Years Old Girls 3-9 Boys 7 Students I D...,6岁以上的儿童玩具小女孩 3-9 男孩 7 学生I日 学生I日 10 儿童8 婴儿4 拼图
2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg,47598,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex between men and wom...,sm qing qu alternative sex men women metal tai...,sm qing qu alternative sex men women metal tai...,tail men fox ting alternative qu sex sai women...,tail men fox metal alternative qu sex sai wome...,sm qing qu Alternative Sex between Men and Wom...,between Men and Women with Metal Tail Fox Tail...,Alternative Sex between Men and Women with Met...,使用金属尾狐尾巴的男女替代性行为
3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg,36608,traditional chinese painting burnin' up yingke...,traditional chinese painting burnin up yingkes...,traditional chinese painting burnin yingkesong...,traditional chinese painting burnin yingkesong...,lucky patron paintings decorative traditional ...,lucky patron paintings decorative traditional ...,Traditional Chinese Painting Burnin' Up Yingke...,Yingkesong Landscape Painting Patron Living Ro...,Burnin' Up Yingkesong Landscape Painting Patro...,中华传统绘画烧在延克松露地绘画主办人 客厅绘画老板办公室 幸运装饰墙壁
4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg,45566,lgnace lee men's jeans thick section distresse...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lgnace lee mens jeans thick section distressed...,lee jeans mens tide brand slacksmusic lgnace t...,lee jeans mens tide brand slacksmusic lgnace t...,lgnace Lee Men's Jeans Thick Section Distresse...,Section Distressed Straight Youth Autumn Tide ...,Men's Jeans Thick Section Distressed Straight ...,Lgnace Lee Lee Men的Jeans Thick分会受困于直立青年 秋秋潮的B...


In [70]:
image_embed_path = "./image_embeds/val_set_embed"
text_embed_path = "./text_embeds/val_set_embed"
image_embed_files = os.listdir(image_embed_path)
text_embed_files = os.listdir(text_embed_path)

In [71]:
all_image_embed ={}
all_text_embed = {}
for i,t in tqdm(zip(image_embed_files,text_embed_files)):
    with open(f"{image_embed_path}/{i}","rb") as img_file:
        img_embed = pickle.load(img_file)
        all_image_embed[img_embed["model_name"]] = img_embed["embed"]
    with open(f"{text_embed_path}/{t}","rb") as txt_file:
        txt_embed = pickle.load(txt_file)
        all_text_embed[txt_embed["model_name"]] = txt_embed["embed"]


11it [00:08,  1.26it/s]


In [72]:
all_image_embed.keys()

dict_keys(['large_v2_e_5', 'base_5th_e_5', 'base_all_e_7', 'base_2nd_e_5', 'base_1st_e_5', 'large_v1_e_5', 'base_3rd_e_5', 'base_4th_e_5', 'large_v3_e_1', 'large_all_e_9', 'large_v4_e_1'])

In [73]:
all_text_embed.keys()

dict_keys(['large_v4_e_1', 'large_v2_e_5', 'base_1st_e_5', 'base_3rd_e_5', 'base_2nd_e_5', 'base_all_e_7', 'base_5th_e_5', 'large_all_e_9', 'large_v3_e_1', 'base_4th_e_5', 'large_v1_e_5'])

In [74]:
pair_to_test = "large_all_e_9"

In [75]:
# embed_to_ensamble = ["base_1st_e_5","base_2nd_e_5"]

# embed_to_ensamble = list(all_text_embed.keys())

In [76]:
def get_ensamble(all_image_embed,all_text_embed,embeds_to_ensamble,ensamble_strategy,all_weights):
    ensamsables = {}
    img_embed_list = []
    text_embed_list = []
    all_weights = all_weights
    for model_name ,weights in zip(embeds_to_ensamble,range(len(all_weights))):
        img_embed_list.append(all_image_embed[model_name]*all_weights[weights])
        text_embed_list.append(all_text_embed[model_name]*all_weights[weights])
    if len(embeds_to_ensamble) > 1:
        if ensamble_strategy =="concat":
            ensamsables["image"] = torch.concat(img_embed_list,dim=1)
            ensamsables["text"] = torch.concat(text_embed_list,dim=1)
        elif ensamble_strategy =="avg":
            ensamsables["image"] = torch.mean(torch.stack(img_embed_list),0)
            ensamsables["text"] = torch.mean(torch.stack(text_embed_list),0)
        else:
            ensamsables["image"] = img_embed_list
            ensamsables["text"] = text_embed_list
        return ensamsables
    else:
            ensamsables["image"] = img_embed_list[0]
            ensamsables["text"] = text_embed_list[0]
            return ensamsables


        


In [77]:
import itertools
# test = itertools.combinations(all_text_embed,3)

In [78]:

def get_preds(ensamsables,score=False,df=None,use_faiss=True,auto_faiss=False,max_ensamble=False,sharpen_weight = 1):
    image_embed = ensamsables["image"]
    text_embed = ensamsables["text"]
    if use_faiss:
        faiss_index = faiss.IndexFlatIP(image_embed.shape[1])
        faiss_index.add(image_embed.numpy().astype(np.float32))
        top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
        top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
    elif auto_faiss:
        auto_faiss_index,index_infos  = build_index(image_embed.numpy().astype(np.float32), save_on_disk=False)
        top5_k_e,top5_k_y_pred = auto_faiss_index.search(text_embed.numpy().astype(np.float32),5)
        top10_k_e,top10_k_y_pred = auto_faiss_index.search(text_embed.numpy().astype(np.float32),10)

    elif max_ensamble:
        all_top10_cosine_sim  =[]
        all_top5_cosine_sim  =[]
        all_top10_pred_idx  =[]
        all_top5_pred_idx  =[]
        for text,image in zip(text_embed,image_embed):
            faiss_index = faiss.IndexFlatIP(image.shape[1])
            faiss_index.add(image.numpy().astype(np.float32))
            top5_cosine_sim,top5_pred_idx = faiss_index.search(text.numpy().astype(np.float32),5)
            top10_cosine_sim,top10_pred_idx = faiss_index.search(text.numpy().astype(np.float32),10)
            all_top10_cosine_sim.append(top10_cosine_sim)
            all_top5_cosine_sim.append(top5_cosine_sim) 
            all_top10_pred_idx.append(top10_pred_idx)
            all_top5_pred_idx.append(top5_pred_idx)
        all_top10_cosine_sim  = np.concatenate(all_top10_cosine_sim,axis=1)
        all_top5_cosine_sim  = np.concatenate(all_top5_cosine_sim,axis=1)
        all_top10_pred_idx  = np.concatenate(all_top10_pred_idx,axis=1)
        all_top5_pred_idx  = np.concatenate(all_top5_pred_idx,axis=1)
        top5_k_y_pred = []
        top10_k_y_pred = []
        for t5,t10,t5_i,t10_i in zip(all_top5_cosine_sim,all_top10_cosine_sim,all_top5_pred_idx,all_top10_pred_idx):
            top5_k_y_pred.append(t5_i[np.argpartition(t5,-5)[-5:]])
            top10_k_y_pred.append(t10_i[np.argpartition(t10,-10)[-10:]])
        print(top10_k_y_pred)

    else:
        all_cosine_sim  =[]
        all_weights = [0.3,0.5,0.2]
        for text,image,w_i in zip(text_embed,image_embed,range(len(all_weights))):
            cosine_sim = text @ image.T 
            all_cosine_sim.append(cosine_sim * all_weights[w_i])
            del cosine_sim
        # ensamble_cosine_sim = torch.mean(torch.stack(all_cosine_sim),dim=0) 
        # ensamble_cosine_sim = 1 / torch.mean(1. / (torch.stack(all_cosine_sim)) +0.0001,dim=0) 
        ensamble_cosine_sim = torch.prod(torch.stack(all_cosine_sim),dim=0) 
        top5_k_e,top5_k_y_pred = torch.topk(ensamble_cosine_sim,5)
        top10_k_e,top10_k_y_pred = torch.topk(ensamble_cosine_sim,10)
    if score:
            top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
            top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
            top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
            top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
            avg_acc = (top5_acc+top10_acc)/2
            return top5_acc,top10_acc,avg_acc
    else:
        return top5_k_y_pred,top10_k_y_pred

In [79]:
strategy = ["concat"]
# embeds_to_score = [["base_5th_e_5","large_v2_e_5","large_v3_e_1","large_v1_e_5","large_v4_e_1"]] + [i for c in range(2,6) for i in itertools.combinations(["base_5th_e_5","large_v2_e_5","large_v3_e_1","large_v1_e_5","large_v4_e_1"],c)]
# embeds_to_score =[["base_5th_e_5","large_v2_e_5","large_v3_e_1","large_v1_e_5","large_v4_e_1"]]
embeds_to_score = [[key] for key in list(all_text_embed.keys())]
score_df = pd.DataFrame()
embeds_to_score_name = []
top5_score = []
top10_score = []
avg_score = []
for s in strategy:
    for e in tqdm(embeds_to_score):
        ensamsables = get_ensamble(all_image_embed,all_text_embed,e,s,[1,1,1,1,1])
        top5_acc,top10_acc,avg_acc = get_preds(ensamsables,score=True,use_faiss=True,auto_faiss=False,max_ensamble=False,df=clean_df)
        if len(e) == len(list(all_text_embed.keys())):
            name = "all models"
        elif len(e) >1 and  len(e) < len(list(all_text_embed.keys())):
            name = " + ".join(e)
        else:
            name = e[0]
        embeds_to_score_name.append(name)
        top5_score.append(top5_acc)
        top10_score.append(top10_acc)
        avg_score.append(avg_acc)
score_df["model"] = embeds_to_score_name
score_df["top_5_acc"] = top5_score
score_df["top_10_acc"] = top10_score
score_df["avg_acc"] = avg_score

score_df

100%|██████████| 11/11 [01:02<00:00,  5.65s/it]


,model,top_5_acc,top_10_acc,avg_acc
0,large_v4_e_1,0.54818,0.67414,0.61116
1,large_v2_e_5,0.56708,0.69000,0.62854
2,base_1st_e_5,0.50582,0.63248,0.56915
3,base_3rd_e_5,0.50848,0.63554,0.57201
4,base_2nd_e_5,0.50958,0.63606,0.57282
5,base_all_e_7,0.48374,0.61216,0.54795
6,base_5th_e_5,0.51670,0.64624,0.58147
7,large_all_e_9,0.52332,0.64678,0.58505
8,large_v3_e_1,0.54694,0.67216,0.60955
9,base_4th_e_5,0.51768,0.64680,0.58224


In [80]:
score_df.sort_values("avg_acc",ascending=False)

,model,top_5_acc,top_10_acc,avg_acc
1,large_v2_e_5,0.56708,0.69000,0.62854
0,large_v4_e_1,0.54818,0.67414,0.61116
10,large_v1_e_5,0.54804,0.67272,0.61038
8,large_v3_e_1,0.54694,0.67216,0.60955
7,large_all_e_9,0.52332,0.64678,0.58505
9,base_4th_e_5,0.51768,0.64680,0.58224
6,base_5th_e_5,0.51670,0.64624,0.58147
4,base_2nd_e_5,0.50958,0.63606,0.57282
3,base_3rd_e_5,0.50848,0.63554,0.57201
2,base_1st_e_5,0.50582,0.63248,0.56915


In [81]:
# score_df.to_csv("combination_results_8_6_22.csv",index=False)

In [87]:
score_df = pd.read_csv("combination_results_8_6_22.csv")
score_df.head()

,model,top_5_acc,top_10_acc,avg_acc
0,large_v2_e_5,0.56708,0.69000,0.62854
1,base_1st_e_5,0.50582,0.63248,0.56915
2,base_3rd_e_5,0.50848,0.63554,0.57201
3,base_2nd_e_5,0.50958,0.63606,0.57282
4,base_all_e_7,0.48374,0.61216,0.54795


In [89]:
pd.set_option('max_colwidth', 400)

score_df.sort_values("avg_acc",ascending=False).head(10)

,model,top_5_acc,top_10_acc,avg_acc
69,large_v2_e_5 + base_5th_e_5 + large_v1_e_5,0.59548,0.71570,0.65559
317,large_v2_e_5 + base_2nd_e_5 + base_5th_e_5 + large_all_e_9 + large_v1_e_5,0.59438,0.71440,0.65439
171,large_v2_e_5 + base_2nd_e_5 + base_5th_e_5 + large_v1_e_5,0.59270,0.71446,0.65358
72,large_v2_e_5 + base_4th_e_5 + large_v1_e_5,0.59274,0.71330,0.65302
324,large_v2_e_5 + base_5th_e_5 + large_all_e_9 + base_4th_e_5 + large_v1_e_5,0.59290,0.71256,0.65273
62,large_v2_e_5 + base_2nd_e_5 + large_v1_e_5,0.59232,0.71256,0.65244
67,large_v2_e_5 + base_5th_e_5 + large_all_e_9,0.59252,0.71184,0.65218
287,large_v2_e_5 + base_1st_e_5 + base_5th_e_5 + large_all_e_9 + large_v1_e_5,0.59118,0.71182,0.65150
319,large_v2_e_5 + base_2nd_e_5 + large_all_e_9 + base_4th_e_5 + large_v1_e_5,0.59152,0.71146,0.65149
182,large_v2_e_5 + base_5th_e_5 + large_all_e_9 + large_v1_e_5,0.59166,0.71116,0.65141


In [82]:
# strategy = ["max"]
# # embeds_to_score = [[key] for key in list(all_text_embed.keys())] + [i for c in range(2,9) for i in itertools.combinations(all_text_embed,c)]
# embeds_to_score =[["base_4th_e_5","large_v2_e_5","large_v1_e_5"]]
# embeds_to_score_name = []
# c = []
# top10_preds = []
# top10_preds_score = []

# for s in strategy:
#     for e in embeds_to_score:
#         ensamsables = get_ensamble(all_image_embed,all_text_embed,e,s)
#         top10_score,top10_acc = get_preds(ensamsables,score=False,use_faiss=False,max_ensamble=True,auto_faiss=False,df=clean_df)
#         if len(e) == len(list(all_text_embed.keys())):
#             name = "all models"
#         elif len(e) >1 and  len(e) < len(list(all_text_embed.keys())):
#             name = " + ".join(e)
#         else:
#             name = e[0]
#         embeds_to_score_name.append(name)
#         top10_preds.append(top10_acc)
#         top10_preds_score.append(top10_score)




In [83]:
# top5_pred,top10_pred =remap_pred_to_name(top5_pred_idx,top10_pred_idx,clean_df["product"].values)

In [84]:
# def results_to_json(top5_pred,top10_pred,caption_array,result_dir,result_file_name):
#     results = []
#     for c,t5,t10 in tqdm(zip(caption_array,top5_pred,top10_pred)):
#         result_dict = {}
#         result_dict["caption"] = c
#         result_dict["top5"] = t5
#         result_dict["top10"] = t10
#         results.append(result_dict)
#     print("checking results format")
#     print(results[0])
#     with open(f"{result_dir}/{result_file_name}.json","w") as results_file:
#         json.dump(results,results_file)
#     with open(f"{result_dir}/{result_file_name}.json","r") as results_file:
#         check_results = json.load(results_file)
#     print("checking results format of saved file")
#     print(check_results[0])
#     return check_results
    
        



In [85]:
# saved_result = results_to_json(top5_pred,top10_pred,clean_df["caption"].values,"./submissions","test_val_ensamble")